20190610: keras_lr_finder is not updated in pip, certain functions are misbehaving

20190609: image quality and sample size may not be enough to mess around with

In [0]:
import os
import numpy as np
import keras
from keras.models import Sequential, save_model, load_model
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras.preprocessing.image import *
from keras.applications.vgg16 import VGG16, preprocess_input

Using TensorFlow backend.


Mount my Google Drive folder

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

# setting some useful path variables
path_proj = 'gdrive/My Drive/Projects_2019/Brain_MRI_Images_for_Brain_Tumor_Detection/'
path_model = path_proj + 'Model/'
path_data = path_proj + 'Data/'
path_data_yes = path_proj + 'Data/yes/'
path_data_no = path_proj + 'Data/no/'

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


Load from my Google Drive folder

In [0]:
# train:val is set to be 8:2
imgen = ImageDataGenerator(validation_split=0.2)
X_trn = imgen.flow_from_directory(path_data, batch_size=203, target_size=(256,256), subset='training', shuffle=True)
X_val = imgen.flow_from_directory(path_data, batch_size=50, target_size=(256,256), subset='validation', shuffle=True)

Found 203 images belonging to 2 classes.
Found 50 images belonging to 2 classes.


In [0]:
# Load the VGG16 to be our feature extractor
V16 = VGG16(weights='imagenet', include_top=False, input_shape=(256,256,3))

# Freeze the layers of our feature extractor, we'll only train the classifier (fully-connected layers) and use the VGG16 directly.
for layer in V16.layers:
    layer.trainable = False

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
# Just to make sure model is clean before adding layers to it
model = None
model = Sequential()
# "Borrow" VGG16 as a feature extractor
model.add(V16)
# Concatenate the feature extracted from VGG16
model.add(Flatten())
#model.add(Dense(256, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(16, activation='relu'))
#model.add(Dropout(0.2))
model.add(Dense(8, activation='relu'))
model.add(Dropout(0.2))
#model.add(Dense(4, activation='relu'))
#model.add(Dropout(0.2))
# Softmax layer for classification
model.add(Dense(2, activation='softmax'))

# Checking the model architecture
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 8, 8, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               4194432   
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 8)                 264       
__________

Learning rate finder for Keras
Package downloaded via Pip is not updated, so I just copied the links here

https://github.com/surmenok/keras_lr_finder/blob/master/keras_lr_finder/lr_finder.py

https://towardsdatascience.com/estimating-optimal-learning-rate-for-a-deep-neural-network-ce32f2556ce0

In [0]:
# If there's no improvement for 5 epochs, then training will stop
ES = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10, min_delta=0.01)
# Saving the best model during training
CP = keras.callbacks.ModelCheckpoint(path_model + 'vgg16_checkpoint.hdf5', save_best_only=True)

In [0]:
# Specify the optimiser, initial learning rate, loss function and metrics
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=['acc'])

In [0]:
# Train the model
history = model.fit_generator(X_trn, steps_per_epoch=1, epochs=9999, validation_data=X_val, validation_steps=1, callbacks=[ES, CP])
print('checkpoint is saved as: ' + path_model + 'vgg16_checkpoint.hdf5')

Epoch 1/9999
1/1 [==============================] - 3s 3s/step - loss: 2.6349 - acc: 0.8128 - val_loss: 2.6950 - val_acc: 0.8200
Epoch 2/9999
1/1 [==============================] - 2s 2s/step - loss: 3.0126 - acc: 0.8030 - val_loss: 2.4638 - val_acc: 0.8200
Epoch 3/9999
1/1 [==============================] - 2s 2s/step - loss: 2.9055 - acc: 0.7931 - val_loss: 2.2617 - val_acc: 0.8200
Epoch 4/9999
1/1 [==============================] - 2s 2s/step - loss: 2.7684 - acc: 0.8030 - val_loss: 2.1054 - val_acc: 0.8600
Epoch 5/9999
1/1 [==============================] - 2s 2s/step - loss: 2.8419 - acc: 0.8079 - val_loss: 1.9220 - val_acc: 0.8400
Epoch 6/9999
1/1 [==============================] - 2s 2s/step - loss: 3.6716 - acc: 0.7488 - val_loss: 1.8471 - val_acc: 0.8200
Epoch 7/9999
1/1 [==============================] - 2s 2s/step - loss: 2.7909 - acc: 0.8128 - val_loss: 1.8172 - val_acc: 0.8200
Epoch 8/9999
1/1 [==============================] - 2s 2s/step - loss: 2.7161 - acc: 0.7980 - val

In [0]:
# Load the best model during training
model = load_model(path_model + 'vgg16_checkpoint.hdf5')

In [4]:
!echo "# MRI_brain_tumour_binary" >> README.md
!git config --global user.name 'mrken823'
!git init
!git add README.md
!git commit -m "first commit"
!git remote add origin https://github.com/mrken823/MRI_brain_tumour_binary.git
!git push -u origin master

Reinitialized existing Git repository in /content/.git/

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@fe17336fe706.(none)')
fatal: remote origin already exists.
error: src refspec master does not match any.
error: failed to push some refs to 'https://github.com/mrken823/MRI_brain_tumour_binary.git'
